In [1]:
# import useful libraries

import numpy as np
import itertools
from collections import Counter
from utils import load_puzzle_input, test_code
import ast
import re
from tqdm import tqdm
import math

In [2]:
# load puzzle input
    
puzzle_input = load_puzzle_input('input.txt')

print(puzzle_input[:3])

['Monkey 0:', '  Starting items: 66, 71, 94', '  Operation: new = old * 5']


### Part 1

In [3]:
# code for preparing and testing examples for part 1

example_dict_part_1 = {
    tuple(load_puzzle_input('example_1.txt')): 10605
}

In [4]:
def parse_puzzle_input(puzzle_input):
    
    return {
        i: {
            'items': [int(x) for x in puzzle_input[(i * 7) + 1][18:].split(',')], 
            'operation_operation': puzzle_input[(i * 7) + 2][23:24], 
            'operation_x': puzzle_input[(i * 7) + 2][25:], # could be int, could be 'old'
            'division_test': int(puzzle_input[(i * 7) + 3][21:]), 
            'if_true': int(puzzle_input[(i * 7) + 4][-1]), 
            'if_false': int(puzzle_input[(i * 7) + 5][-1]),
            'inspect_count': 0
            } for i in range((len(puzzle_input) // 7) + 1)
        }  

In [5]:
def perform_operation(value, operation_operation, operation_x):
    x = value if operation_x == 'old' else int(operation_x)
    
    if operation_operation == '*':
        return value * x
    elif operation_operation == '+':
        return value + x

In [6]:
def solve_part_1(puzzle_input):

    monkey_dict = parse_puzzle_input(puzzle_input)

    # print(monkey_dict)

    for _ in range(20):

        for monkey_id in monkey_dict:

            monkey_dict[monkey_id]['inspect_count'] += len(monkey_dict[monkey_id]['items'])

            for item in monkey_dict[monkey_id]['items']:

                new_value = perform_operation(
                    item,
                    monkey_dict[monkey_id]['operation_operation'],
                    monkey_dict[monkey_id]['operation_x'] 
                    )

                new_value = int(new_value / 3) # monkey boredom

                if not new_value % monkey_dict[monkey_id]['division_test']: #i.e. it IS divisible
                    monkey_dict[monkey_dict[monkey_id]['if_true']]['items'].append(new_value)
                else:
                    monkey_dict[monkey_dict[monkey_id]['if_false']]['items'].append(new_value)

            monkey_dict[monkey_id]['items'] = []

    monkey_activity_list = [monkey_dict[id]['inspect_count'] for id in monkey_dict]
    monkey_activity_list.sort()
    return np.prod(monkey_activity_list[-2:])


In [7]:
# test part 1 solution

test_code(solve_part_1, example_dict_part_1)

Test 0 passed: Input <('Monkey 0:', '  Starting items: 79, 98', '  Operation: new = old * 19', '  Test: divisible by 23', '    If true: throw to monkey 2', '    If false: throw to monkey 3', '', 'Monkey 1:', '  Starting items: 54, 65, 75, 74', '  Operation: new = old + 6', '  Test: divisible by 19', '    If true: throw to monkey 2', '    If false: throw to monkey 0', '', 'Monkey 2:', '  Starting items: 79, 60, 97', '  Operation: new = old * old', '  Test: divisible by 13', '    If true: throw to monkey 1', '    If false: throw to monkey 3', '', 'Monkey 3:', '  Starting items: 74', '  Operation: new = old + 3', '  Test: divisible by 17', '    If true: throw to monkey 0', '    If false: throw to monkey 1')> gives output <10605>.

Congratulations! Looks like you cracked it! Good job!


In [8]:
solve_part_1(puzzle_input)

55944

### Part 2

In [9]:
# code for preparing and testing examples for part 2

example_dict_part_2 = {
    tuple(load_puzzle_input('example_1.txt')): 2713310158
}

In [16]:
a = 25
a %= 5
a

0

In [31]:
def find_common_multiple(monkey_dict):
    # lcm = 1
    # for monkey_id in monkey_dict:
    #     lcm *= monkey_dict[monkey_id]['division_test']
    # return lcm
    return math.lcm(*[monkey_dict[monkey_id]['division_test'] for monkey_id in monkey_dict])

In [55]:
def solve_part_2(puzzle_input):

    monkey_dict = parse_puzzle_input(puzzle_input)

    # find a number that we can always modulo the 'worry' by
    # so that it doesn't change the way that the other modulos work
    common_multiple = find_common_multiple(monkey_dict)

    for _ in tqdm(range(10000)): # should be 10000 in the real thing

        for monkey_id in monkey_dict:

            monkey_dict[monkey_id]['inspect_count'] += len(monkey_dict[monkey_id]['items'])

            for item in monkey_dict[monkey_id]['items']:

                new_value = perform_operation(
                    item,
                    monkey_dict[monkey_id]['operation_operation'],
                    monkey_dict[monkey_id]['operation_x'] 
                    )

                # no more monkey boredom, instead we modulo by our common multiple
                new_value = new_value % common_multiple

                if new_value < 0:
                    print(new_value)
                    print(monkey_id)
                    print(item)
                    break

                if not new_value % monkey_dict[monkey_id]['division_test']: #i.e. it IS divisible
                    monkey_dict[monkey_dict[monkey_id]['if_true']]['items'].append(new_value)
                else:
                    monkey_dict[monkey_dict[monkey_id]['if_false']]['items'].append(new_value)

            monkey_dict[monkey_id]['items'] = []

    monkey_activity_list = [monkey_dict[id]['inspect_count'] for id in monkey_dict]
    monkey_activity_list.sort()
    # print(monkey_activity_list)
    # return np.prod(monkey_activity_list[-2:]) # well this sucks. np prod gives a negative result even though I got the question right...
    return math.prod(monkey_activity_list[-2:])


In [56]:
# test part 2 solution

test_code(solve_part_2, example_dict_part_2)

100%|██████████| 10000/10000 [00:00<00:00, 102040.52it/s]

Test 0 passed: Input <('Monkey 0:', '  Starting items: 79, 98', '  Operation: new = old * 19', '  Test: divisible by 23', '    If true: throw to monkey 2', '    If false: throw to monkey 3', '', 'Monkey 1:', '  Starting items: 54, 65, 75, 74', '  Operation: new = old + 6', '  Test: divisible by 19', '    If true: throw to monkey 2', '    If false: throw to monkey 0', '', 'Monkey 2:', '  Starting items: 79, 60, 97', '  Operation: new = old * old', '  Test: divisible by 13', '    If true: throw to monkey 1', '    If false: throw to monkey 3', '', 'Monkey 3:', '  Starting items: 74', '  Operation: new = old + 3', '  Test: divisible by 17', '    If true: throw to monkey 0', '    If false: throw to monkey 1')> gives output <2713310158>.

Congratulations! Looks like you cracked it! Good job!


In [57]:
solve_part_2(puzzle_input)

100%|██████████| 10000/10000 [00:00<00:00, 27624.64it/s]


15117269860